## Experiment 1: 

In [ ]:
import argparse
import csv
import os

import numpy as np
import scipy.sparse as ssp
import torch
import torch.nn.functional as F
import time

from matplotlib import pyplot as plt
from yacs.config import CfgNode

from utils.ogbdataset import loaddataset
from utils.heuristic import CN, AA, RA
from models.GNN import GAT_Variant, GCN_Variant, SAGE_Variant, GIN_Variant, GAE_forall, InnerProduct, mlp_score


: 

## Create GNNs 

In [ ]:
def create_GAE_model(cfg_model: CN,
                     cfg_score: CN,
                     model_name: str):
    if model_name in {'GAT', 'VGAE', 'GAE', 'GraphSage'}:
        raise NotImplementedError('Current model does not exist')
        # model = create_model(cfg_model)

    elif model_name == 'GAT_Variant':
        encoder = GAT_Variant(cfg_model.in_channels,
                              cfg_model.hidden_channels,
                              cfg_model.out_channels,
                              cfg_model.num_layers,
                              cfg_model.dropout,
                              cfg_model.heads,
                              )
    elif model_name == 'GCN_Variant':
        encoder = GCN_Variant(cfg_model.in_channels,
                              cfg_model.hidden_channels,
                              cfg_model.out_channels,
                              cfg_model.num_layers,
                              cfg_model.dropout,
                              )
    elif model_name == 'SAGE_Variant':
        encoder = SAGE_Variant(cfg_model.in_channels,
                               cfg_model.hidden_channels,
                               cfg_model.out_channels,
                               cfg_model.num_layers,
                               cfg_model.dropout,
                               )
    elif model_name == 'GIN_Variant':
        encoder = GIN_Variant(cfg_model.in_channels,
                              cfg_model.hidden_channels,
                              cfg_model.out_channels,
                              cfg_model.num_layers,
                              cfg_model.dropout,
                              cfg_model.mlp_layer
                              )
    if cfg_score.product == 'dot':
        decoder = mlp_score(cfg_model.out_channels,
                            cfg_score.score_hidden_channels,
                            cfg_score.score_out_channels,
                            cfg_score.score_num_layers,
                            cfg_score.score_dropout,
                            cfg_score.product)
    elif cfg_score.product == 'inner':
        decoder = InnerProduct()

    else:
        # Without this else I got: UnboundLocalError: local variable 'model' referenced before assignment
        raise ValueError('Current model does not exist')

    return GAE_forall(encoder=encoder, decoder=decoder)


### Analysis Tool 

In [ ]:
def save_to_csv(file_path, model_name, heuristic, test_loss):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    file_exists = os.path.isfile(file_path)
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(['Model', 'Heuristic', 'Test_Loss'])
        writer.writerow([model_name, heuristic, test_loss])
def visualize(pred, true_label, save_path = './visualization.png'):

    pred = pred.cpu().detach().numpy()
    true_label = true_label.cpu().detach().numpy()
    plt.figure(figsize=(10, 6))
    plt.scatter(np.arange(len(true_label)), true_label, color='blue', label='True Score', alpha=0.6)
    plt.scatter(np.arange(len(pred)), pred, color='red', label='Prediction', alpha=0.6)

    plt.title('Predictions vs True Score')
    plt.xlabel('Sample Index')
    plt.ylabel('Value')
    plt.ylim(0, 1.5)
    plt.legend()

    plt.savefig(save_path)
    plt.close()

    print(f"Visualization saved at {save_path}")


In [ ]:
def train(model, optimizer, data, splits, device, epoch):
    model.train()
    optimizer.zero_grad()

    # Positive and negative edges for training
    pos_edge_index = splits['train']['pos_edge_label_index'].to(device)
    neg_edge_index = splits['train']['neg_edge_label_index'].to(device)

    # Labels for positive and negative edges (continuous regression labels)
    pos_edge_label = splits['train']['pos_edge_score'].to(device)
    neg_edge_label = splits['train']['neg_edge_score'].to(device)

    # Forward pass
    z = model.encode(data.x, data.edge_index)

    # Compute predictions for both positive and negative edges
    pos_pred = model.decode(z[pos_edge_index[0]], z[pos_edge_index[1]])
    neg_pred = model.decode(z[neg_edge_index[0]], z[neg_edge_index[1]])

    # Compute regression loss (MSE for continuous labels)
    pos_loss = F.mse_loss(pos_pred, pos_edge_label)
    neg_loss = F.mse_loss(neg_pred, neg_edge_label)
    loss = pos_loss + neg_loss
    loss.backward()

    # Optimizer step
    optimizer.step()
    visualize(pos_pred, pos_edge_label, save_path='./visualization_pos_train.png')
    visualize(neg_pred, neg_edge_label, save_path='./visualization_neg_train.png')

    return loss.item()



@torch.no_grad()
def valid(model, data, splits, device, epoch):
    model.eval()

    # Positive and negative edges for validation
    pos_edge_index = splits['valid']['pos_edge_label_index'].to(device)
    neg_edge_index = splits['valid']['neg_edge_label_index'].to(device)

    # Labels for positive and negative edges (continuous regression labels)
    pos_edge_label = splits['valid']['pos_edge_score'].to(device)
    neg_edge_label = splits['valid']['neg_edge_score'].to(device)

    # Forward pass
    z = model.encode(data.x, data.edge_index)

    # Predict scores for both positive and negative edges
    pos_pred = model.decode(z[pos_edge_index[0]], z[pos_edge_index[1]])
    neg_pred = model.decode(z[neg_edge_index[0]], z[neg_edge_index[1]])

    # Compute regression loss (MSE)
    pos_loss = F.mse_loss(pos_pred, pos_edge_label)
    neg_loss = F.mse_loss(neg_pred, neg_edge_label)
    loss = pos_loss + neg_loss


    return loss.item()


@torch.no_grad()
def test(model, data, splits, device):
    model.eval()

    # Positive and negative edges for test
    pos_edge_index = splits['test']['pos_edge_label_index'].to(device)
    neg_edge_index = splits['test']['neg_edge_label_index'].to(device)

    # Labels for positive and negative edges (continuous regression labels)
    pos_edge_label = splits['test']['pos_edge_score'].to(device)
    neg_edge_label = splits['test']['neg_edge_score'].to(device)

    # Forward pass
    z = model.encode(data.x, data.edge_index)

    # Predict scores for both positive and negative edges
    pos_pred = model.decode(z[pos_edge_index[0]], z[pos_edge_index[1]])
    neg_pred = model.decode(z[neg_edge_index[0]], z[neg_edge_index[1]])
    visualize(pos_pred, pos_edge_label, save_path = './visualization_pos.png')
    visualize(neg_pred, neg_edge_label, save_path = './visualization_neg.png')

    # Compute regression loss (MSE)
    pos_loss = F.mse_loss(pos_pred, pos_edge_label)
    neg_loss = F.mse_loss(neg_pred, neg_edge_label)
    loss = pos_loss + neg_loss

    return loss.item()



In [ ]:
# def parseargs():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--epochs', type=int, default=30, help="number of epochs")
#     parser.add_argument('--dataset', type=str, default="Cora")
#     parser.add_argument('--batch_size', type=int, default=512, help="batch size")
#     parser.add_argument('--heuristic', type=str, default="AA")
#     parser.add_argument('--gnn', type=str, default="gcn")
#     parser.add_argument('--model', type=str, default="GIN_Variant")
#     parser.add_argument('--use_feature', type=bool, default=False)
#     args = parser.parse_args()
#     return args

class Config:
    def __init__(self):
        self.epochs = 30
        self.dataset = "Cora"
        self.batch_size = 512
        self.heuristic = "AA"
        self.gnn = "gcn"
        self.model = "GIN_Variant"
        self.use_feature = False


In [ ]:

args = Config()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, splits = loaddataset(args.dataset, True)
data = data.to(device)
with open('./yamls/cora/heart_gnn_models.yaml', "r") as f:
    cfg = CfgNode.load_cfg(f)
cfg_model = eval(f'cfg.model.{args.model}')
if not hasattr(splits['train'], 'x') or splits['train'].x is None:
    cfg_model.in_channels = 1024
else:
    cfg_model.in_channels = data.num_nodes
cfg_score = eval(f'cfg.score.{args.model}')
cfg.model.type = args.model
edge_weight = torch.ones(data.edge_index.size(1), dtype=float)
A = ssp.csr_matrix(
    (edge_weight, (data.edge_index[0].cpu(), data.edge_index[1].cpu())),
    shape=(data.num_nodes, data.num_nodes)
)
method_dict = {
    "CN": CN,
    "AA": AA,
    "RA": RA
}
for split in splits:
    pos_edge_score, _ = method_dict[args.heuristic](A, splits[split]['pos_edge_label_index'],
                                                    batch_size=args.batch_size)
    neg_edge_score, _ = method_dict[args.heuristic](A, splits[split]['neg_edge_label_index'],
                                                    batch_size=args.batch_size)
    splits[split]['pos_edge_score'] = torch.sigmoid(pos_edge_score)
    splits[split]['neg_edge_score'] = torch.sigmoid(neg_edge_score)
if not args.use_feature:
    A_dense = A.toarray()
    A_tensor = torch.tensor(A_dense)
    data.x = A_tensor.float().to(device)

model = create_GAE_model(cfg_model, cfg_score, args.model).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, args.epochs + 1):
    start = time.time()
    loss = train(model, optimizer, data, splits, device, args.batch_size)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
test_loss = test(model, data, splits, device)
save_to_csv(f'./results/test_results_{args.dataset}.csv', args.model, args.heuristic, test_loss)
print('Saved results.')

